In [1]:
#<node pkg="assignment_2_2023" type="action_client.py" name="action_client" launch-prefix="xterm -e"/>
%matplotlib widget
import rospy
from geometry_msgs.msg import Point, Pose, Twist
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import Odometry
import math
import actionlib
import assignment_2_2023.msg
from assignment_2_2023.msg import Status
from tf import transformations
from std_srvs.srv import *
import time
from actionlib_msgs.msg import GoalStatus
import ipywidgets as widgets

import matplotlib.pyplot as plt
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

In [2]:
#global variables for graphics
global sendGoal, goalCancelled
global nReachedGoals, nSentGoals, nCancelledGoals
global x_goal, y_goal
sendGoal, goalCancelled = False, False
nReachedGoals, nSentGoals, nCancelledGoals = 0, 0, 0
x_goal,y_goal = 0.0, 0.0



# return true if the goal has been reached by the robot, false otherwise
def goalReached():
    """
    Function which returns true if the goal has been reached by the robot, false otherwise
    """
    return client.get_state() == actionlib.GoalStatus.SUCCEEDED



# Callback of subscriber '/odom', custom message 'Status' is used as a publisher
def callback(msg):
    """
    Callback function of subscriber '/odom'

    Args:
        msg: position and velocity of the robot
    """
    
    status = Status()
    status.x = msg.pose.pose.position.x
    status.y = msg.pose.pose.position.y
    status.v_x = msg.twist.twist.linear.x
    status.v_z = msg.twist.twist.angular.z
    pub.publish(status)
    

In [3]:
"""
This is the main code of the action client.
"""
# initialize node action_client
rospy.init_node('action_client')

# define publisher '/status/
global pub
pub = rospy.Publisher("/status", Status, queue_size=1)

# subscriber odom
rospy.Subscriber("/odom", Odometry, callback)



msg = "start"

global client    
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
client.wait_for_server()



True

In [4]:
# Analizes the coordinates and sends the goal
def sendGoalEvent(x,y):

    global sendGoal, nSentGoals
    global x_goal, y_goal
    
    msg = ""
    limit = 10
    
    # x and y must be in the interval [-limit, limit], otherwise an exception is generated
    if x<-limit or x>limit or y<-limit or y>limit:
        msg = f"Insert values between [{-limit}, {limit}]"
        
    x_goal, y_goal = x,y


    # instance of the goal
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y

    # set x,y as ROS parameters
    rospy.set_param('des_pos_x', x)
    rospy.set_param('des_pos_y', y)
    
    
    sendGoal = True
    nSentGoals = nSentGoals+1
    
    # send the goal
    client.send_goal(goal)
    msg = f"Goal ({x}, {y}) sent!"
    return msg




# Analizes the coordinates and sends the goal
def cancelGoalEvent():
    global nCancelledGoals    
    global goalCancelled

    msg = ""
    # if the goal is reached, you can't cancel it
    if goalReached():
        msg = "Goal already reached, you cannot cancel it"
    else:
        # cancel the goal
        try:
            client.cancel_goal()
            msg = "Goal cancelled"
            
            goalCancelled = True
            nCancelledGoals = nCancelledGoals+1
    
        except Exception as e:
            msg = str(e)
    return msg

In [5]:
#class to visualize the data on the graphics
class Visualiser:
    global nReachedGoals, nSentGoals, nCancelledGoals    
    global x_goal, y_goal

    def __init__(self):
        self.fig, self.ax = plt.subplots(2, 1, figsize=(7,12))
        self.ln, = self.ax[0].plot([], [], 'ro', markersize=3)
        self.x_data, self.y_data = [], []
        self.new_target = False
        sendGoal = False        

        self.variables = np.array(["Reached Goals", "Sent Goals", "Cancelled Goals"])
        self.bars = self.ax[1].bar(self.variables, [0, 0, 0])
        self.ax[1].set_ylim(0, 10)

    def plot_init(self):
        self.ax[0].set_xlim(-10, 10)
        self.ax[0].set_ylim(-10, 10)
        return self.ln, self.bars

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)     
        
        
        #After 5000 it removes the first element of the position lists
        if len(self.y_data) > 5000:# Limiting the length of data for efficiency
            self.y_data.pop(0)
            self.x_data.pop(0)


    def update_plot(self, frame):  
        global sendGoal, nReachedGoals

        self.ln.set_data(self.x_data, self.y_data)
        
        if sendGoal:        
            self.new_target = True
            
            if goalReached():        
                sendGoal = False
                nReachedGoals = nReachedGoals+1
        
        #it draws the target
        if self.new_target:
            self.ax[0].scatter(x_goal, y_goal, marker='+', color='blue', s=50)  # Draw the blue cross marker at the position (X_pos, Y_pos)
            self.new_target = False
            
        # Update bar plot setting the heights
        self.bars[0].set_height(nReachedGoals)
        self.bars[1].set_height(nSentGoals)
        self.bars[2].set_height(nCancelledGoals)



In [6]:

#numerical box, X cordinate goal
x_wid = widgets.BoundedFloatText(
    min=-10.0,
    max=10.0,
    step=0.5,
    description='X:',
    disabled=False)

#numerical box, Y cordinate goal
y_wid = widgets.BoundedFloatText(
    min=-10.0,
    max=10.0,
    step=0.5,
    description='Y:',
    disabled=False)

display(x_wid,y_wid)


#Button send goal
sendGoal_button = widgets.Button(description="Send Goal!") 
output_sendGoal = widgets.Output()
display(sendGoal_button, output_sendGoal)


#Button cancel goal
cancelGoal_button = widgets.Button(description="Cancel Goal!") 
output_cancelGoal = widgets.Output()
display(cancelGoal_button, output_cancelGoal)



#TextBox, "disbled" = false se voglio scrivere sopra
textBox = widgets.Text(value=msg, disabled=True)
display(textBox)




#handle changes, the observe method of the widget can be used to register a callback.
def sendGoal_button_clicked(b):
    msg = ""
    global goalCancelled
    with output_sendGoal:
        try:
            x = x_wid.value
            y = y_wid.value
            
            msg = sendGoalEvent(x,y)
            goalCancelled = False
                            
        except Exception as e:
            msg = "Invalid Input"
        
    textBox.value = msg        
    print(msg)
        
    



#handle changes, the observe method of the widget can be used to register a callback.
def cancelGoal_button_clicked(b):
    msg = ""
    global goalCancelled
    with output_cancelGoal:
        if(goalCancelled == False):
            msg = cancelGoalEvent()
            
            goalCancelled = True
        else:
            msg = "Goal already cancelled"
        
    textBox.value = msg
    print(msg)
    

sendGoal_button.on_click(sendGoal_button_clicked)
cancelGoal_button.on_click(cancelGoal_button_clicked)

    
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit = True)
plt.show(block=True)

BoundedFloatText(value=0.0, description='X:', max=10.0, min=-10.0, step=0.5)

BoundedFloatText(value=0.0, description='Y:', max=10.0, min=-10.0, step=0.5)

Button(description='Send Goal!', style=ButtonStyle())

Output()

Button(description='Cancel Goal!', style=ButtonStyle())

Output()

Text(value='start', disabled=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Goal (0.0, 0.0) sent!
